## Setup

In [ ]:
import sys
from pathlib import Path
import cv2
import numpy as np
from IPython.display import Video, Image as IPImage

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

print("Setup complete!")
print(f"Working directory: {Path.cwd()}")

## Step 1: Data Preprocessing (Dry-Run)

Process just the first video from each split for quick testing.

In [ ]:
%%bash
# Run data pipeline in dry-run mode (processes only first video per split)
python ../src/data_pipeline.py --dry-run --fps 1 --min-area 100

# Check output
echo ""
echo "Dataset created at: datasets/merged/"
echo "Config created at: configs/dataset_video.yaml"

## Step 2: Inspect Dataset

Let's look at a sample image and its YOLO label.

In [ ]:
from data_pipeline import mask_to_yolo_boxes
import matplotlib.pyplot as plt

# Find a sample image
merged_dir = Path("../datasets/merged")
train_images = list((merged_dir / "images" / "train").glob("*.jpg"))

if train_images:
    sample_img = train_images[0]
    sample_label = merged_dir / "labels" / "train" / f"{sample_img.stem}.txt"
    
    # Read image
    img = cv2.imread(str(sample_img))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]
    
    # Read labels
    if sample_label.exists():
        with open(sample_label, 'r') as f:
            labels = [line.strip().split() for line in f.readlines()]
        
        # Draw boxes
        for label in labels:
            cls, x_center, y_center, width, height = map(float, label)
            
            # Convert to pixel coordinates
            x1 = int((x_center - width/2) * w)
            y1 = int((y_center - height/2) * h)
            x2 = int((x_center + width/2) * w)
            y2 = int((y_center + height/2) * h)
            
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Display
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.title(f"Sample: {sample_img.name}")
    plt.axis('off')
    plt.show()
    
    print(f"Image: {sample_img.name}")
    print(f"Labels: {len(labels) if sample_label.exists() else 0} boxes")
else:
    print("No training images found. Run data pipeline first.")

## Step 3: Train Model (Quick Test)

Train for just 5 epochs as a quick test.

In [ ]:
%%bash
# Quick training run (5 epochs)
python ../src/train_yolo.py --model yolov8n.pt --epochs 5 --batch 8 --imgsz 320

echo ""
echo "Training complete!"
echo "Weights saved to: runs/detect/video_merged/weights/best.pt"

## Step 4: Run Demo Inference

Run inference on a test video and generate outputs.

In [ ]:
# Check if we have a test video
test_videos_dir = Path("../datasets/Video_dataset/test/rgb")
sample_videos = list(test_videos_dir.glob("*.mp4")) if test_videos_dir.exists() else []

if sample_videos:
    test_video = sample_videos[0]
    print(f"Found test video: {test_video.name}")
else:
    # Use any available video
    print("No test video found. Place a video at datasets/samples/my_road_video.mp4")
    test_video = None

In [ ]:
if test_video:
    !python ../src/run_demo.py \
        --weights ../runs/detect/video_merged/weights/best.pt \
        --video {test_video} \
        --output ../outputs \
        --conf 0.25
    
    print("\nDemo inference complete!")
    print("Check outputs/ directory for results")
else:
    print("Skipping demo - no test video available")

## Step 5: View Results

In [ ]:
import json
import pandas as pd

# Load JSON report
output_dir = Path("../outputs")
json_files = list(output_dir.glob("*_report.json"))

if json_files:
    with open(json_files[0], 'r') as f:
        report = json.load(f)
    
    print("=" * 50)
    print("INFERENCE SUMMARY")
    print("=" * 50)
    print(f"Total frames: {report['summary']['total_frames']}")
    print(f"Total detections: {report['summary']['total_detections']}")
    print(f"Unique objects: {report['summary']['unique_objects']}")
    print(f"Avg detections/frame: {report['summary']['avg_detections_per_frame']:.2f}")
    print("=" * 50)
else:
    print("No report found. Run demo inference first.")

In [ ]:
# Display first frame with detections
first_frames = list(output_dir.glob("*_first_frame.jpg"))

if first_frames:
    first_frame = cv2.imread(str(first_frames[0]))
    first_frame = cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(14, 10))
    plt.imshow(first_frame)
    plt.title("First Frame with Detections")
    plt.axis('off')
    plt.show()
else:
    print("No first frame image found.")

In [ ]:
# Load and display CSV summary
csv_files = list(output_dir.glob("*_summary.csv"))

if csv_files:
    df = pd.read_csv(csv_files[0])
    
    print("\nSummary DataFrame (first 10 rows):")
    print(df.head(10))
    
    # Plot detections over time
    plt.figure(figsize=(12, 4))
    plt.plot(df['frame_idx'], df['num_detections'], linewidth=2)
    plt.xlabel('Frame Index')
    plt.ylabel('Number of Detections')
    plt.title('Detections Over Time')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No CSV summary found.")

## Step 6: Display Output Video

View the annotated output video.

In [ ]:
# Find output video
output_videos = list(output_dir.glob("*_out.mp4"))

if output_videos:
    print(f"Output video: {output_videos[0].name}")
    print("\nNote: Video playback may not work in all environments.")
    print(f"View the video directly at: {output_videos[0]}")
    
    # Try to display (may not work in all environments)
    try:
        Video(str(output_videos[0]), embed=True, width=800)
    except:
        print("Could not embed video. Open it manually.")
else:
    print("No output video found.")

## Next Steps

1. **Full Training**: Train with more epochs for better performance
   ```bash
   python src/train_yolo.py --epochs 50 --batch 16
   ```

2. **Auto Fine-tuning**: Improve model with pseudo-labeling
   ```bash
   python src/auto_finetune.py --weights runs/detect/video_merged/weights/best.pt --unlabeled datasets/samples/
   ```

3. **Process Full Dataset**: Remove `--dry-run` flag
   ```bash
   python src/data_pipeline.py
   ```